In [1]:
import pandas as pd
from pathlib import Path

from ast import literal_eval
import re

import myvariant
import time

import numpy as np

from flaml import AutoML

In [2]:
DATA_DIRECTORY = Path("/Users/kevin/projects/ezancestry/data/aisnps")


In [3]:
kg = pd.read_csv(DATA_DIRECTORY.joinpath("thousand_genomes.kidd.dataframe.csv"))
# read the header line to get column names
with open(DATA_DIRECTORY.joinpath("kidd.aisnp.1kg.vcf")) as f:
    for line in f:
        if line.startswith("#CHROM"):
            colnames = line.strip().split("\t")
            break
kgvcf = pd.read_csv(DATA_DIRECTORY.joinpath("kidd.aisnp.1kg.vcf"), sep="\t", comment="#", header=None, names=colnames)


In [4]:
# In the original DataFrame, 55 positions (records) and 2513 samples (columns)
kgvcf.shape


(55, 2513)

In [5]:
# process the columns
kgvcf.drop(columns=["QUAL", "FILTER", "INFO", "FORMAT"], inplace=True)
kgvcf.set_index(["#CHROM", "POS", "REF", "ALT"], inplace=True)


In [6]:
# unique values for snps
pd.unique(kgvcf.drop(columns=["ID"]).values.ravel("K"))


array(['0|0', '0|1', '1|1', '1|0'], dtype=object)

In [7]:
# set the index as rsid actuall
kgvcf.rename(columns={"ID": "rsid"}, inplace=True)
kgvcf.set_index(["rsid"], inplace=True)


In [8]:
dragen = pd.read_csv(DATA_DIRECTORY.joinpath("dragen.kidd.dataframe.csv"))


In [9]:
# dragen_index = dragen.set_index(["chrom", "pos", "ref", "alt"]).index
dragen.set_index(["chrom", "pos", "ref", "alt"], inplace=True)


In [10]:
def parse_genotypes(longstr):
    longstr = longstr.replace("gts", "'gts'")
    longstr = longstr.replace("=", ":")
    longstr = re.sub(r"id:([a-zA-Z0-9_.-]*)", r"'id':'\1'", longstr)
    return literal_eval(longstr)


In [11]:
dragen["genotypes"] = dragen["samples"].apply(parse_genotypes)
dragen.drop(columns=["samples"], inplace=True)


In [12]:
# pd.concat(dragen["genotypes"].apply(pd.DataFrame).to_list(), keys=dragen.index).reset_index()
dragen_gts = pd.concat(dragen["genotypes"].apply(pd.DataFrame).to_list(), keys=dragen.index)
dragen_gts = dragen_gts.droplevel(4)


In [13]:
# the index of dragen_gts has unique chrom, pos, ref, alt
dragen_gts.head()


id     gts
chrom pos      ref alt                 
chr20 63528151 T   C    HG03300  [1, 1]
                   C    HG03799  [0, 1]
                   C    HG03190  [0, 1]
                   C    HG03352  [1, 1]
                   C    NA20281  [0, 1]

In [14]:
def ref_alt_to_gts(row):
    # ref = row["ref"]
    # alt = row["alt"]
    _, _, ref, alt = row.name
    gtsring = ""
    for gt in row["gts"]:
        if gt:
            gtsring += ref
        else:
            gtsring += alt
    return gtsring


In [15]:
# convert the 0,1 to ref alt
dragen_gts["new_gts"] = dragen_gts.apply(ref_alt_to_gts, axis=1)

# pivot
# dragen_gts = dragen_gts.pivot(columns="id", values="new_gts")
dragen_gts.reset_index(inplace=True)
dragen_gts = dragen_gts.pivot(index=['chrom', 'pos', 'ref', 'alt'], columns='id', values='new_gts')
# dragen_gts = dragen_gts.pivot(columns="id", values="gts")


In [16]:
def list_to_string(gt):
    try:
        return "|".join([str(_) for _ in gt])
    except:
        return np.nan


In [17]:
dragen_gts = dragen_gts.applymap(list_to_string)


In [18]:
# snp positions are the records, samples are the columns
dragen_gts.shape


(55, 3202)

In [19]:
mv = myvariant.MyVariantInfo()


In [20]:
def annotate(row):
    chrom = row["chrom"]
    pos = row["pos"]
    ref = row["ref"]
    alt = row["alt"]

    variant = mv.getvariant(f"{chrom}:g.{pos}{ref}>{alt}", assembly="hg38", fields=["dbsnp"])
    time.sleep(0.25)
    return variant["dbsnp"]["rsid"]


In [21]:
dragen_gts_ = dragen_gts.reset_index()
dragen_gts_.head()


id,chrom,pos,ref,alt,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,chr1,101244007,T,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,C|T,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,151150013,C,T,T|C,C|C,T|C,C|C,T|C,C|C,...,C|C,T|C,C|C,C|C,C|C,C|C,C|C,C|C,C|C,T|C
2,chr1,159204893,T,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr10,93161308,A,G,G|A,NaN,NaN,G|A,NaN,NaN,...,G|A,NaN,G|A,G|A,G|A,NaN,NaN,G|A,G|A,G|A
4,chr11,61829740,C,T,NaN,NaN,NaN,T|C,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# now we have the rsid to compare
try:
    dragen_gts_ = pd.read_csv(DATA_DIRECTORY.joinpath("dragen_gts.csv"))
except FileNotFoundError:
    print("no dragen_gts.csv")
    dragen_gts_["rsid"] = dragen_gts_.apply(annotate, axis=1)
    dragen_gts_.to_csv(DATA_DIRECTORY.joinpath("dragen_gts.csv"), index=False)


In [23]:
# use the rsid as index
dragen_gts = dragen_gts_.set_index(["rsid"])


In [24]:
def apply_refref(row):
     refref = row["ref"] + "|" + row["ref"]
     return row.fillna(refref)

In [25]:
dragen_gts = dragen_gts.apply(apply_refref, axis=1)

In [26]:
from ezancestry.process import get_1kg_labels

In [27]:
# only train and evaluate on the samples in the 1kg data
dfsamples = get_1kg_labels()

original_samples = set(dfsamples.index)

# make sure the 1kg vcf (DataFrame) only has samples listed in the sample index
kgsamples = set(kgvcf.columns.to_list())
kgsamples = kgsamples.intersection(original_samples)

# inner join the 1kg and dragen samples
dragensamples = set(dragen_gts.columns.to_list())
dragensamples = dragensamples.intersection(kgsamples)

In [28]:
len(dragensamples) == len(kgsamples)

True

In [29]:
dragendf = dragen_gts[dragensamples].T.copy()

In [30]:
dragendf.apply(lambda row: sorted(row.replace("|", "")))

rsid,rs3737576,rs7554936,rs2814778,rs4918664,rs174570,rs1079597,rs2238151,rs671,rs7997709,rs1572018,...,rs16891982,rs7722456,rs192655,rs3823159,rs917115,rs1462906,rs6990312,rs2196051,rs1871534,rs3814134
NA20814,C|T,C|C,C|T,A|A,C|C,C|C,C|T,A|G,C|C,C|T,...,C|C,C|C,A|G,A|A,C|T,C|T,G|G,A|A,C|C,A|A
NA20859,C|T,C|C,C|T,A|A,C|C,C|C,C|T,A|G,C|C,C|T,...,C|C,C|C,A|G,A|A,C|T,C|T,G|G,A|A,C|C,A|A
HG01125,C|T,C|C,C|T,A|A,C|C,C|C,C|T,A|G,C|C,C|T,...,C|C,C|C,A|G,A|A,C|T,C|T,G|G,A|A,C|C,A|A
HG02301,C|T,C|C,C|T,A|A,C|C,C|C,C|T,A|G,C|C,C|T,...,C|C,C|C,A|G,A|A,C|T,C|T,G|G,A|A,C|C,A|A
HG04015,C|T,C|C,C|T,A|A,C|C,C|C,C|T,A|G,C|C,C|T,...,C|C,C|C,A|G,A|A,C|T,C|T,G|G,A|A,C|C,A|A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA19740,T|T,T|C,T|T,G|A,T|C,T|C,T|T,G|G,T|C,T|T,...,G|C,T|C,G|G,G|A,T|T,T|T,T|G,G|A,G|C,G|A
NA19777,T|T,T|C,T|T,G|A,T|C,T|C,T|T,G|G,T|C,T|T,...,G|C,T|C,G|G,G|A,T|T,T|T,T|G,G|A,G|C,G|A
NA20298,T|T,T|C,T|T,G|A,T|C,T|C,T|T,G|G,T|C,T|T,...,G|C,T|C,G|G,G|A,T|T,T|T,T|G,G|A,G|C,G|A
HG03380,T|T,T|C,T|T,G|A,T|C,T|C,T|T,G|G,T|C,T|T,...,G|C,T|C,G|G,G|A,T|T,T|T,T|G,G|A,G|C,G|A


In [31]:
# we're actually going to use this DataFrame instead of the vcf because it has the alleles
kgdf = pd.read_csv(DATA_DIRECTORY.joinpath("thousand_genomes.kidd.dataframe.csv"))
kgdf.rename(columns={"Unnamed: 0": "id"}, inplace=True)
kgdf.drop(columns=["population", "superpopulation", "gender"])
kgdf.set_index("id", inplace=True)
kgdf = kgdf.loc[kgsamples].copy()

# to match dragendf
kgdf.columns.name = "rsid"

In [32]:
# replace | with empty
dragendf = dragendf.apply(lambda row: row.str.replace("|", "", regex=False))

In [33]:
kgdf = kgdf.reindex(dfsamples.index)
dragendf = dragendf.reindex(dfsamples.index)

In [34]:
(dragendf.index == kgdf.index).all()

True

In [35]:
# across the 55 sites, how many different allele combinations in 1kg dataset?
kgdf.nunique().sum()

198

In [36]:
# across the 55 sites, how many different allele combinations in dragen dataset?
dragendf.nunique().sum()

115

# There's more variation in the 1kg than in dragen

In [37]:
kgdf["rs3737576"].value_counts()

TT    2147
CT     293
CC      64
Name: rs3737576, dtype: int64

In [38]:
# where did the homs go in this snp?
dragendf["rs3737576"].value_counts()

TT    2212
CT     292
Name: rs3737576, dtype: int64

In [39]:
# even in the original dragen dataframe they are missing
dragen_gts.loc["rs3737576"].drop(["chrom", "pos", "ref", "alt"]).value_counts()

T|T    2830
C|T     372
Name: rs3737576, dtype: int64

# Model fitting

In [40]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display="diagram")


In [41]:
# rearrange the columns in kgdf to match dragendf
kgdf = kgdf[dragendf.columns]

In [42]:
dragendf_categories = dragendf.apply(pd.unique).values.tolist()
kgdf_categories = kgdf.apply(lambda col: col.unique()).T.values

kgdf_categories = [a.tolist() for a in kgdf_categories]
dragendf_categories = [a.tolist() for a in dragendf_categories]

In [43]:
categorical_transformer = OneHotEncoder(sparse=False, drop="first")
categorical_columns = dragendf.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_columns),
    ]
)

In [44]:
automl_kg  = AutoML()

In [46]:
# define the pipeline first

pipe_kg = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("pca", PCA(n_components=3)),
        ("automl", automl_kg),
    ]
)

In [47]:
pipe_kg

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['rs3737576', 'rs7554936',
                                                   'rs2814778', 'rs4918664',
                                                   'rs174570', 'rs1079597',
                                                   'rs2238151', 'rs671',
                                                   'rs7997709', 'rs1572018',
                                                   'rs2166624', 'rs7326934',
                                                   'rs9522149', 'rs200354',
                                                   'rs1800414', 'rs12913832',
                                                   'rs12439433', 'rs735480',
                                                   'rs1426654', 'rs459920',
                                                   'rs...
                        learner_selector='sample', log_file_name='',
                        log_training_metric=False, log_type='better',
                        max_iter=None, mem_thres=4294967296, metric='auto',
                        metric_constraints=[], min_sample_size=10000,
                        model_history=False, n_concurrent_trials=1, n_jobs=-1,
                        n_splits=5, pred_time_limit=inf, retrain_full=True,
                        sample=True, split_ratio=0.1, split_type='auto',
                        starting_points='static', task='classification', ...))])

In [48]:
X_dragen = dragendf.copy()
X_kg = kgdf.copy()
y = dfsamples["superpopulation"].copy()

In [49]:
settings = {
    "automl__time_budget": 600, # 10 min  # total running time in seconds
    "automl__metric": 'log_loss',  # primary metrics can be chosen from: ['accuracy','roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'f1','log_loss','mae','mse','r2']
    "automl__task": 'classification',  # task type  
    "automl__estimator_list":["rf", "kneighbor", 'xgboost','lgbm'],
    "automl__eval_method": "cv",
    "automl__log_file_name": 'kg_experiment.log',  # flaml log file
}
pipe_kg.fit(X_kg, y, **settings)

[flaml.automl: 06-26 05:51:53] {2390} INFO - task = classification
[flaml.automl: 06-26 05:51:53] {2392} INFO - Data split method: stratified
[flaml.automl: 06-26 05:51:53] {2396} INFO - Evaluation method: cv
[flaml.automl: 06-26 05:51:53] {2465} INFO - Minimizing error metric: log_loss
[flaml.automl: 06-26 05:51:53] {2605} INFO - List of ML learners in AutoML Run: ['rf', 'kneighbor', 'xgboost', 'lgbm']
[flaml.automl: 06-26 05:51:53] {2897} INFO - iteration 0, current learner rf
[flaml.automl: 06-26 05:51:54] {3026} INFO - Estimated sufficient time budget=12064s. Estimated necessary time budget=21s.
[flaml.automl: 06-26 05:51:54] {3078} INFO -  at 1.2s,	estimator rf's best error=0.6602,	best estimator rf's best error=0.6602
[flaml.automl: 06-26 05:51:54] {2897} INFO - iteration 1, current learner lgbm
[flaml.automl: 06-26 05:51:54] {3078} INFO -  at 1.3s,	estimator lgbm's best error=0.9167,	best estimator rf's best error=0.6602
[flaml.automl: 06-26 05:51:54] {2897} INFO - iteration 2, 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['rs3737576', 'rs7554936',
                                                   'rs2814778', 'rs4918664',
                                                   'rs174570', 'rs1079597',
                                                   'rs2238151', 'rs671',
                                                   'rs7997709', 'rs1572018',
                                                   'rs2166624', 'rs7326934',
                                                   'rs9522149', 'rs200354',
                                                   'rs1800414', 'rs12913832',
                                                   'rs12439433', 'rs735480',
                                                   'rs1426654', 'rs459920',
                                                   'rs...
                        learner_selector='sample', log_file_name='',
                        log_training_metric=False, log_type='better',
                        max_iter=None, mem_thres=4294967296, metric='auto',
                        metric_constraints=[], min_sample_size=10000,
                        model_history=False, n_concurrent_trials=1, n_jobs=-1,
                        n_splits=5, pred_time_limit=inf, retrain_full=True,
                        sample=True, split_ratio=0.1, split_type='auto',
                        starting_points='static', task='classification', ...))])

In [50]:
automl_kg = pipe_kg.named_steps["automl"]

In [51]:
automl_kg.best_estimator

'xgboost'

In [52]:
automl_kg

AutoML(append_log=False, auto_augment=True, custom_hp={}, early_stop=False,
       ensemble=False, estimator_list='auto', eval_method='auto',
       fit_kwargs_by_estimator={}, hpo_method='auto', keep_search_state=False,
       learner_selector='sample', log_file_name='', log_training_metric=False,
       log_type='better', max_iter=None, mem_thres=4294967296, metric='auto',
       metric_constraints=[], min_sample_size=10000, model_history=False,
       n_concurrent_trials=1, n_jobs=-1, n_splits=5, pred_time_limit=inf,
       retrain_full=True, sample=True, split_ratio=0.1, split_type='auto',
       starting_points='static', task='classification', ...)

In [53]:
automl_kg.best_config

{'n_estimators': 171,
 'max_leaves': 4,
 'min_child_weight': 0.06997546880412316,
 'learning_rate': 0.07175365925500445,
 'subsample': 0.6131164292538739,
 'colsample_bylevel': 0.7463027987383976,
 'colsample_bytree': 0.9778796034251762,
 'reg_alpha': 0.030103799938277834,
 'reg_lambda': 0.49806412606274414}

In [54]:
automl_kg.best_loss_per_estimator

{'rf': 0.289698456964712,
 'kneighbor': 0.29335653755719804,
 'xgboost': 0.28041827195162544,
 'lgbm': 0.28233096811501507}

In [55]:
automl_kg.best_loss

0.28041827195162544

# Dragen

In [57]:
automl_dragen  = AutoML()

In [58]:
pipe_dragen = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("pca", PCA(n_components=3)),
        ("automl", automl_dragen),
    ]
)

In [59]:
settings = {
    "automl__time_budget": 600, # 10 min  # total running time in seconds
    "automl__metric": 'log_loss',  # primary metrics can be chosen from: ['accuracy','roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'f1','log_loss','mae','mse','r2']
    "automl__task": 'classification',  # task type  
    "automl__estimator_list":["rf", "kneighbor", 'xgboost','lgbm'],
    "automl__eval_method": "cv",
    "automl__log_file_name": 'dragen_experiment.log',  # flaml log file
}
pipe_dragen.fit(X_dragen, y, **settings)

[flaml.automl: 06-26 06:05:54] {2390} INFO - task = classification
[flaml.automl: 06-26 06:05:54] {2392} INFO - Data split method: stratified
[flaml.automl: 06-26 06:05:54] {2396} INFO - Evaluation method: cv
[flaml.automl: 06-26 06:05:54] {2465} INFO - Minimizing error metric: log_loss
[flaml.automl: 06-26 06:05:54] {2605} INFO - List of ML learners in AutoML Run: ['rf', 'kneighbor', 'xgboost', 'lgbm']
[flaml.automl: 06-26 06:05:54] {2897} INFO - iteration 0, current learner rf
[flaml.automl: 06-26 06:05:55] {3026} INFO - Estimated sufficient time budget=12259s. Estimated necessary time budget=21s.
[flaml.automl: 06-26 06:05:55] {3078} INFO -  at 1.3s,	estimator rf's best error=0.8566,	best estimator rf's best error=0.8566
[flaml.automl: 06-26 06:05:55] {2897} INFO - iteration 1, current learner lgbm
[flaml.automl: 06-26 06:05:55] {3078} INFO -  at 1.4s,	estimator lgbm's best error=1.0454,	best estimator rf's best error=0.8566
[flaml.automl: 06-26 06:05:55] {2897} INFO - iteration 2, 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['rs3737576', 'rs7554936',
                                                   'rs2814778', 'rs4918664',
                                                   'rs174570', 'rs1079597',
                                                   'rs2238151', 'rs671',
                                                   'rs7997709', 'rs1572018',
                                                   'rs2166624', 'rs7326934',
                                                   'rs9522149', 'rs200354',
                                                   'rs1800414', 'rs12913832',
                                                   'rs12439433', 'rs735480',
                                                   'rs1426654', 'rs459920',
                                                   'rs...
                        learner_selector='sample', log_file_name='',
                        log_training_metric=False, log_type='better',
                        max_iter=None, mem_thres=4294967296, metric='auto',
                        metric_constraints=[], min_sample_size=10000,
                        model_history=False, n_concurrent_trials=1, n_jobs=-1,
                        n_splits=5, pred_time_limit=inf, retrain_full=True,
                        sample=True, split_ratio=0.1, split_type='auto',
                        starting_points='static', task='classification', ...))])

In [60]:
automl_dragen = pipe_dragen.named_steps["automl"]

In [61]:
automl_dragen.best_estimator

'lgbm'

In [62]:
automl_dragen.best_config

{'n_estimators': 86,
 'num_leaves': 4,
 'min_child_samples': 34,
 'learning_rate': 0.08150699535290234,
 'log_max_bin': 5,
 'colsample_bytree': 0.9318718192262867,
 'reg_alpha': 0.025787729278819956,
 'reg_lambda': 0.3474746756955603}

In [63]:
automl_dragen.best_loss_per_estimator

{'rf': 0.5130578397211927,
 'kneighbor': 0.5427940615919921,
 'xgboost': 0.5120057862976479,
 'lgbm': 0.5071824051664984}

# After 10 minutes of searching for the best model, the log loss for best 1kg model is better than the best dragen model.
# I bumped the time_limit for training to 30 min and the pattern is consistent.

In [64]:
automl_kg.best_loss_per_estimator

{'rf': 0.289698456964712,
 'kneighbor': 0.29335653755719804,
 'xgboost': 0.28041827195162544,
 'lgbm': 0.28233096811501507}

In [65]:
automl_dragen.best_loss_per_estimator

{'rf': 0.5130578397211927,
 'kneighbor': 0.5427940615919921,
 'xgboost': 0.5120057862976479,
 'lgbm': 0.5071824051664984}